In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
class GoogleMapsScraper:
    def __init__(self,headless=False,restaurants=True,hotels=True):
        self.headless=headless
        self.restaurants=restaurants
        self.hotels=hotels
        self.RestaurantData=[]
        self.HotelsData=[]
        options=Options()
        options.add_experimental_option("detach", True)
        if headless:
            options.headless=True
        else:
            options.headless=False

        self.driver=webdriver.Chrome(ChromeDriverManager().install(),options=options)
        
    def SearchMaps(self,location):
     
        
        #first page
        self.driver.get("https://www.google.com/maps/search/"+str(location))
        
        

        try:
            #If google maps is unable to search the place
            NotFound=self.driver.find_element_by_xpath("//*[@class='IPum6b']")
            return "Search Not found"     #try improving your search keywords
        
        except:
            #if place is searched on google maps 
            try:
                #check if number of locations matching keyword is displayes on google maps or a single location is foud
                
                #if number of locations found select the first one
                FirstLocation=self.driver.find_element_by_xpath("//a[@class='hfpxzc']")
                FirstLocation.click()
                FirstLocationName=str((self.driver.find_element_by_xpath("//*[@class='hfpxzc']")).get_attribute("aria-label"))
                #open the location in google maps
                self.driver.get("https://www.google.com/maps/search/"+FirstLocationName)

            except:
                pass
            
            
            
                
            if self.restaurants:
                
                #wait till the restaurant button on maps load
                element = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//button[@aria-label='Restaurants']")))
                #click the restaurant button to get nearest restaurants 
                self.driver.find_element_by_xpath("//button[@aria-label='Restaurants']").click()
              
                #wait for restaurants data to load 
                element = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//*[@class='hfpxzc']")))
                
                #scroll 2 times to load first 12 restaurants. "initially data of only 4 is loaded"
                for i in range(0,2):
                    scroll=self.driver.find_elements_by_xpath("//a[@jsan='7.hfpxzc,0.aria-label,8.href,0.jsaction']")
                    scroll[-1].send_keys(Keys.END)
                    
                #fetch resturants data an store in rest
                fetched_restaurants=self.driver.find_elements_by_xpath("//*[@class='Nv2PK THOPZb CpccDe']")

                #iterate over all restaurants data and store its name and google maps link
                for res in fetched_restaurants:
                    self.RestaurantData.append({res.get_attribute("aria-label"):res.get_attribute("href")})
                        
            
            if self.hotels:
                #go back to first page
                self.driver.back()
                #wait till the hotels button on maps load
                element = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//button[@aria-label='Hotels']")))
                #click the restaurant button to get nearest Hotels 
                self.driver.find_element_by_xpath("//button[@aria-label='Hotels']").click()
                
                #wait for Hotels data to load 
                element = WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//*[@class='hfpxzc']")))
                
                #scroll 2 times to load first 12 Hotels. "initially data of only 4 is loaded"
                for i in range(0,2):
                    scroll=self.driver.find_elements_by_xpath("//a[@jsan='7.hfpxzc,0.aria-label,8.href,0.jsaction']")
                    scroll[-1].send_keys(Keys.END)
                    
                #fetch resturants data an store in rest
                fetched_hotels=self.driver.find_elements_by_xpath("//a[@class='hfpxzc']")

                #iterate over all restaurants data and store its name and google maps link
                for hot in fetched_hotels:
                    self.HotelsData.append({hot.get_attribute("aria-label"):hot.get_attribute("href")})
                        
                print(len(self.RestaurantData))
                print(len(self.HotelsData))

                return (self.RestaurantData,self.HotelsData)
            
            
#main
gs=GoogleMapsScraper(restaurants=True,hotels=True)
p=gs.SearchMaps("wadi bani khalid")
print(p)

TypeError: WebDriver.__init__() got multiple values for argument 'options'

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Define Jakarta boundary coordinates
NORTH_EAST = (-6.0969309, 106.9376301)
NORTH_WEST = (-6.0984299, 106.7045931)
SOUTH_WEST = (-6.3546284, 106.7947008)
SOUTH_EAST = (-6.3621171, 106.9130577)

# Set grid step size (smaller steps = more grid points)
LAT_STEP = 0.02
LON_STEP = 0.02

# Initialize Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# Define a function to scroll and collect data
def collect_data():
    stores = []
    try:
        # Scroll to load more data
        for _ in range(10):  # Adjust the range for more scrolling
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
            time.sleep(2)
        
        # Wait for the results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Nv2PK.THOPZb.CpccDe'))
        )
        
        # Collect data
        results = driver.find_elements(By.CSS_SELECTOR, '.Nv2PK.THOPZb.CpccDe')
        for result in results:
            try:
                name = result.find_element(By.CSS_SELECTOR, '.qBF1Pd-font-display-small').text
                address = result.find_element(By.CSS_SELECTOR, '.W4Efsd').text
                stores.append({'Name': name, 'Address': address})
            except:
                continue
    except Exception as e:
        print(f"Error while collecting data: {e}")
    return stores

# Generate grid points and scrape data
data = []
lat = NORTH_WEST[0]
while lat >= SOUTH_WEST[0]:
    lon = NORTH_WEST[1]
    while lon <= NORTH_EAST[1]:
        # Construct the URL for the grid point
        url = f"https://www.google.com/maps/search/frozen+store/@{lat},{lon},14z/"
        print(f"Scraping: {url}")
        driver.get(url)
        time.sleep(5)  # Let the page load
        data.extend(collect_data())
        lon += LON_STEP
    lat -= LAT_STEP

# Close the browser
driver.quit()

# Save data to a CSV file
df = pd.DataFrame(data)
df.to_csv("frozen_stores_jakarta.csv", index=False)
print("Data saved to frozen_stores_jakarta.csv")

Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.7045931,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.72459309999999,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.74459309999999,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.76459309999998,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.78459309999998,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.80459309999998,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.82459309999997,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.84459309999997,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.86459309999996,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.0984299,106.88459309999996,14z/
Scraping: https://www.google.com/maps/search/frozen+store/@-6.09842